In [1]:
import pandas as pd
import numpy as np

Substitute for own data path

In [2]:
df = pd.read_csv(r'C:\Users\eric7\OneDrive\Desktop\aps360Project\convolutional-neural-nuts\data\set.csv')
print("successfully retrieved csv file")

successfully retrieved csv file


IN CSV files, formate as [path:genus]
Created a dictionary that has keys as genus, and items as image paths

In [3]:
grouped = df.groupby('Genus')['Path'].apply(list).to_dict()

mapping = {key: np.array(value) for key, value in grouped.items()}

Analysis of data
- help decide cnn approach
- all data in a numpy dictionary called mapping

In [4]:
print(f"number of classes: {len(mapping)}")

number of classes: 70


In [11]:
LessThan75 = 0
MoreThan75 = 0
for dict_keys in mapping.keys():
    if len(dict_keys) > 75:
        print(f"Class name: {dict_keys}, number of images: {len(dict_keys)}")
        MoreThan75 += 1
    else:
        LessThan75 += 1
print(f"More than 75: {MoreThan75}")
print(f"Less Than 75: {LessThan75}")

More than 75: 0
Less Than 75: 70


In [12]:
#use 10 classes for now
n = 10

CNN model with transfer learning

In [25]:
#imports
#need to pip install torch here
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

In [17]:
class preLearnedFungaV1(nn.Module):
    def __init__(self):
        super(preLearnedFungaV1, self).__init__()
        self.name = "fungaV1"
        self.fc1 = nn.Linear(256*6*6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, n)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = x.squeeze(1)  
        return x

Create training, validation and testing
- apply the transfer learning here

In [21]:
#need to install torchvision
from torch.utils.data import TensorDataset
from torchvision import transforms
#adjust to 224 dimension (input to alexnet)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])

In [22]:
#use alexnet for now
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\eric7/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [02:55<00:00, 1.40MB/s] 


In [26]:
#running this takes a long time, only do it after set.csv is fixed
dict = {}
for key, images in mapping.items():
    dict[key] = []
    for img in images:
        newImg = Image.open(img)
        newImg = transform(newImg)
        feat = alexnet.features(newImg)
        dict[key].append(feat)
# Split data into training, validation, and testing sets
tData, tLabel = [], []
vData, vLabel = [], []
testingData, testingLabel = [], []

#0.75 training, 0.15 valdiation, 0.10 for testing
for key, values in dictionary.items():
    numTrain = int(len(values) * 0.75)
    numVal = int(len(values) * 0.15)

    tLabel.extend([key] * numTrain)
    vLabel.extend([key] * numVal)
    testingLabel.extend([key] * (len(values) - numTrain - numVal))

    tData.extend(values[:numTrain])
    vData.extend(values[numTrain:numTrain + numVal])
    testingData.extend(values[numTrain + numVal:])

tTensor = torch.stack(tData)
vTensor = torch.stack(vData)
testingTensor = torch.stack(testingData)

tLabelTensor = torch.tensor(tLabel)
vLabelTensor = torch.tensor(vLabel)
testingLabelTensor = torch.tensor(testingLabel)

train_set = TensorDataset(tTensor, tLabelTensor)
validation_set = TensorDataset(vTensor, vLabelTensor)
test_set = TensorDataset(testingTensor, testingLabelTensor)

KeyboardInterrupt: 

In [ ]:
#testing if train, validation, and test sets work
print(len(train_set))
print(len(validation_set))
print(len(test_set))

Training code

In [ ]:
def getValidationLoss(net, validation_loader, criterion):
  n = len(validation_loader)
  totalLoss = 0
  for i, data in enumerate(validation_loader):
    inputs, labels = data
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    totalLoss += loss.item()
  totalLoss = totalLoss / n
  return totalLoss

#model name
def get_model_name(name, batch_size, lr, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, lr, epoch)
    return path

#plotting helper function
import matplotlib.pyplot as plt
#plot image function
#modified from lab 2
def plot_training_curve(path):
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    n = len(train_loss)
    #plot model
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [ ]:
allClasses = []
#^need to loop through all classes in mapping (later problem)

In [ ]:
import time

In [ ]:
def train_net_TL(net, batchsize=24, learning_rate=0.007, num_epochs=12):

  torch.manual_seed(1)

  target_classes = allClasses #defined in last part

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.8)

  training_loader, validation_loader, testing_loader = dataLoadFunction(batchsize)

  train_loss_graph = np.zeros(num_epochs)
  val_loss_graph = np.zeros(num_epochs)

  n = len(training_loader)
  startTime = time.time()
  for currepoch in range(num_epochs):
    #print("New epoch")
    trainLoss = 0
    #net.train()
    for i, (inputs, labels) in enumerate(training_loader):
      optimizer.zero_grad()

      inputs = torch.tensor(inputs.detach().numpy()).to(inputs.device)
      outputs = net(inputs)

      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      trainLoss += loss.item()

    #print("Onto validation")
    #net.eval()
    train_loss_graph[currepoch] = trainLoss/n
    val_loss_graph[currepoch] = getValidationLoss(net, validation_loader, criterion)

    print(f"Epoch: {currepoch} Training Loss: {train_loss_graph[currepoch]:.2f}, Validation Loss: {val_loss_graph[currepoch]:.2f}, Time: {time.time() - startTime:.2f} seconds")

    model_path = get_model_name(net.name, batchsize, learning_rate, currepoch)
    torch.save(net.state_dict(), model_path)
  print("Total time: ", time.time()-startTime)

  path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(net.name, batchsize, learning_rate, num_epochs)
  np.savetxt("{}_train_loss.csv".format(path), train_loss_graph)
  np.savetxt("{}_val_loss.csv".format(path), val_loss_graph)

Training here

In [ ]:
net3TL = preLearnedFungaV1()
train_net_TL(net3TL, batchsize=36, num_epochs=20)

In [ ]:
#plot curve here

Test model

In [ ]:
def getAccuracy(net, loader):
  correct, count = 0, 0

  #no gradient modification
  with torch.no_grad():
    print("Total number of batches: ", len(loader))
    for currbatch, (i, corr) in enumerate(loader):
      count += corr.shape[0]
      print("CurrBatch: ", currbatch+1)

      correct += (torch.max(net(i), 1)[1] == corr).sum().item()

  return correct / count